# **Clinical Data Science and Machine Learning with Python**
 
## **Day 1**

- **Instructor**: Teresa Krieger, BIH/Charité (teresa.krieger@charite.de)
- **Course date**: November 2021

**Content**:

1.   References
2.   Basic usage
3.   Importing libraries and data
4.   Data exploration
5.   Performing calculations on data
6.   Visualising data
7.   Dimensional reduction and clustering
8.   Identifying cell types

---
## **1. References**

In this course, we will use Python 3.6 (default in Colab as of February 2021).
The following documentation and links might be useful to you:

- Python 3:
  - https://docs.python.org/3/
- `numpy`:
  - https://numpy.org/doc/stable/user/absolute_beginners.html
- `pandas`:
  - https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html
- `matplotlib`:
  - https://matplotlib.org/stable/
- Source of the PBMC dataset:
  - https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k



---
## **2. Basic usage**

**Jupyter notebook**:

This web-based interface is a **Jupyter notebook**, a web application that allows you to create and share documents which contain executable code, equations, visualizations and explanatory text. 

It allows you to define **cells** of different formats:

- Text cells (like this cell)
- Code cells

You can *run* (execute) each cell seperately by pressing <kbd>Shift</kbd>+<kbd>Enter</kbd> or <kbd>Ctrl</kbd>+<kbd>Enter</kbd>. All names you define (variables, functions...) are available for *both* preceding _and_ following cells once executed.

Some code cells additionally produce an output (text, images, etc.), which will appear underneath the cell.


**Flow control**:

Liker programming languages python allows you to control the flow of variables through your code through conditional statements: `if`, `else`, `for`, `while`.

For example, we could write a script that checks the oxygen saturation of a patient and raises an alert if it goes below a certain threshold:
```python
if oxygen_saturation < 90:
    print("We've got a problem!")
else:
    print("Oxygen levels are looking good.")
```

**Functions**: 

Functions are essentially reusable snippets of code that accept variables as arguments. They can thus perform the same operations on many different inputs. Functions in Python could accept one or several variables, some of which might be optional. 

To learn more about a particular function, you can access its documentation using `help(function_name)`.

```python
# Define a function
def square(number):
    output = number ** 2
    return output

# Use the function
result = square(2)
```

**Methods and attributes**:

Besides functions, Python also knows methods (which are functions attached to variables) and attributes (which are variables attached to variables). These can be accessed using a `.` after the variable name.

For example, a variable that contains `5.7` has a method `is_integer()` and an attribute `real`:


In [ ]:
test_variable = 5.7

In [ ]:
test_variable.is_integer()

In [ ]:
test_variable.real

---
#### **_Your turn_: Exercises**

**Exercise 1**: Try to execute the following cells and make sure the output appears. Also try writing some code yourself!

In [ ]:
print("This statement will appear as output.")
# This is a comment. Comments are ignored in the outputs of a cell.
# print('This statement will NOT appear as output.')

In [ ]:
this_is_a_boolean = False   # A boolean variable is either False or True
print(this_is_a_boolean)

In [ ]:
my_string = "This is a string"
print(my_string)

In [ ]:
# Try your own code below:

**Exercise 2**: When executing the following code, will there be an error? Why (not)?


In [ ]:
x = 5
y = "5"

print(x + y)

**Exercise 3**: What is the expected output of the following code?
```python
x = ""
for i in "Welcome_to_Python!":
    if i == "_":
        x += " "
    else:
        x += i
print(x)
```

**Exercise 4**: Call the documentation for the function `len()`.

---
## **3. Importing libraries and data**

First, we need to install the libraries `scanpy` and `leidenalg` used for RNA sequencing analysis.

In [ ]:
!pip3 install -q scanpy[louvain]
!pip3 install leidenalg
!pip3 install -U matplotlib

Restart the runtime after installing new libraries by clicking on `Runtime > Restart runtime` in the menu bar.

Now we can import the libraries `numpy`, `pandas` and `scanpy` (abbreviated as `np`, `pd` and `sc`).

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

We also need to download our data, which consists of single-cell RNA sequencing data from PBMCs from a healthy donor. It is freely available from 10x Genomics and has been provided for you at the link below. In this code, we use the `!` to execute commands not within this notebook but in the shell (meaning on your computer or, in this case, your colab environment). We also use `wget` to download a file from the internet. 


In [ ]:
!wget -O pbmc.csv https://www.dropbox.com/s/zwiyb1odccjk0cy/pbmc_data.csv?dl=0

We will now use the `read_csv()` function ([see docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)) to load the csv file content as a `DataFrame` into the variable `pbmc`.

In [ ]:
pbmc = pd.read_csv('pbmc.csv', sep=',', index_col=0)

---
### **4. Data exploration**

What does the data in our count matrix look like?

#### `DataFrame` dimensionality

The attribute `shape` contains the number of rows and columns of the table in the form of `(n_rows, n_columns)`.

In [ ]:
pbmc.shape

#### `DataFrame` head/tail

The `head()` and and `tail()` functions can be used to take a look at only the first or last few rows of the table ([see docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html)). 

You can pass a number to these functions to specify how many rows you want to see.

In [ ]:
pbmc.head()  # Shows the first 5 rows (by default)

In [ ]:
pbmc.tail(2)  # Shows the last 2 rows

#### `DataFrame` select entries

To select individual **columns**, we can use the following syntax: 

`data_frame[list_of_columns]`

For example, to select the cells with labels 'AAACATTGATCAGC' and 'AAACCGTGTATGCG', the list of column names of interest could look like this: `list_of_columns = ['AAACATTGATCAGC', 'AAACCGTGTATGCG']`

And putting this together:

In [ ]:
list_of_columns = ['AAACATTGATCAGC', 'AAACCGTGTATGCG']
pbmc[list_of_columns].head()   # Note the use of .head() to show only the first 5 rows

To select individual rows, we can use 'loc' as follows:

`data_frame.loc[list_of_rows, list_of_columns]`

Note that, to select all rows or columns, we just use `:` instead of giving a list. 

So to look at the expression of the gene lysozyme in row 'LYZ' across all columns: 

In [ ]:
pbmc.loc['LYZ', :]   

---
#### **_Your turn_: Exercises**

__Exercise 1__: Get (a) the first 4 rows and (b) the last 5 rows in `pbmc`

**Exercise 2**: Select (a) gene expression for the first 5 genes in the cell labelled 'AAACCGTGCTTCCG' and (b) gene expression of 'S100A9' across all cells.

---
## **5. Performing calculations on data**

Now, we would like to find out **how many counts and how many unique genes** are detected for each cell in our data set. To do this, we need to sum values across columns in our data frame. This is achieved using `sum` in `pandas`. Note that you need to choose which axis to sum over - 0 for rows and 1 for columns:

In [ ]:
total_gene_counts = pbmc.sum(axis=0)  # Sum across rows
total_gene_counts.head()

The code above gives us the total counts detected for each gene across all cells. What if we only want to count how many genes are detected in each cell? Then we first need to determine if a gene is detected, using a boolean, and sum afterwards:

In [ ]:
detected = pbmc>0
detected.head()       # 'True' means that a gene is detected in the cell

In [ ]:
unique_genes = detected.sum(axis=0)
unique_genes.head()

If we want to keep this information for later use or share it with others, we can save our result as a CSV file using the `to_csv` function in `pandas`. Here, we will create a data frame with column labels to ensure that we will remember what our data means, and write it to a file called `unique_genes.csv`.

In [ ]:
df = pd.DataFrame({'Unique_genes': unique_genes})
df.head()

In [ ]:
df.to_csv('unique_genes.csv')

In [ ]:
test = pd.read_csv('unique_genes.csv', sep=',', index_col=0)
test.head()

---
#### **_Your turn_: Exercises**

__Exercise 1__: Instead of determining how many counts are detected in each cell, can you find out how many counts are detected for each gene in our data set? You can store this number in the variable `total_cell_counts`.

**Exercise 2**: Can you also calculate how many cells each individual gene is detected in? Please store this number in the variable `unique_cells`.

**Exercise 3**: Now you can (a) turn your variable `unique_cells` into a `pandas` data frame, (b) save it as a CSV file called `unique_cells.csv` and (c) check that your saved file contains the correct data. 

---
## **6. Visualising data**

To visualise our data, we will use the `pyplot` module from `matplotlib`, which we first need to import. The `%matplotlib inline` command ensures that plots are printed in our notebook.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

We can use a histogram to visualise the number of unique genes detected across cells in our data set:

In [ ]:
plt.hist(unique_genes)
plt.show()

There are many ways to customize this plot, which are give them as arguments to the plotting function. This is true for all `matplotlib` functions. You can find all the options by looking at the documentation of each function, in our case [here](https://https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html).
For example, to plot our data in 20 bins, change the color to green and change the relative width of the bars to 80%, we would use:

In [ ]:
plt.hist(unique_genes, bins=20, color='Green', rwidth=0.8)
plt.show()

And finally, to add a title and axis labels:

In [ ]:
plt.hist(unique_genes, bins=20, color='Green', rwidth=0.8)
plt.title('Unique genes detected per cell')
plt.xlabel('Number of unique genes')
plt.ylabel('Frequency')
plt.show()

Instead of displaying a plot in your notebook, you can also save it to a file using the `.savefig()` method. The file type is determined by the extension you specify in the file name.

As we are working in Colab, we also need to download the file to our computer using `files.download`.

In [ ]:
plt.hist(unique_genes, bins=20, color='Green', rwidth=0.8)
plt.title('Unique genes detected per cell')
plt.xlabel('Number of unique genes')
plt.ylabel('Frequency')
plt.savefig('Histogram_unique_genes.png')

In [ ]:
from google.colab import files
files.download('Histogram_unique_genes.png')

There are lots of different types of plots in `matplotlib`, some of which you can view in a gallery [here](https://matplotlib.org/stable/gallery/index.html) (this will also give you the code used to generate the plots).

For example, we might want to create a scatter plot of the number of unique genes detected vs. the total gene count per cell:


In [ ]:
plt.scatter(x=total_gene_counts, y=unique_genes)
plt.title('Scatter plot')
plt.xlabel('Total gene counts')
plt.ylabel('Number of unique genes')
plt.show()

---
#### **_Your turn_: Exercises**

__Exercise 1__: Plot a histogram to visualise the number of unique cells that a gene is dected in across our data set.

**Exercise 2**: Change the bar colour, axes labels and any other parameters you'd like to try out!

---
## **7. Dimensional reduction and clustering**

For all further steps, we will use the `scanpy` package designed for single-cell RNA sequencing data analysis. It uses a specific format called 'annoted data object' for storing sequencing data and associated metadata. The PBMC dataset has been prepared for you in this format, and we can download it and load it as follows:

In [ ]:
!wget -O pbmc.h5ad https://www.dropbox.com/s/fstt2blgac1nhsi/pbmc.h5ad?dl=0

In [ ]:
adata = sc.read('pbmc.h5ad')

The `scanpy` package contains a lot of inbuilt functions that simplify complex processing steps. For example, to generate the same scatter plot we created above, we can use:

In [ ]:
sc.pl.scatter(adata, x='total_gene_counts', y='unique_genes')

Now we want to run dimensionality reduction and clustering on our data so that we can identify different cell types. First, we need to normalise and scale our data to zero mean and unit variance. We also clip values exceeding 10 standard deviations from the mean.

In [ ]:
sc.pp.scale(adata, max_value=10)

To run principal component analysis (PCA) on our data:

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

To plot different principal components:

In [ ]:
sc.pl.pca(adata, components=['1,2'])

To compute a neighbourhood graph (based on the first 30 principal components and with a neighbourhood size of 10) and the UMAP dimensional reduction, which will help us to better distinguish our cell clusters in 2D:

In [ ]:
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=10)
sc.tl.umap(adata)

Now we can plot gene expression across all cells using the UMAP representation:

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

To perform clustering:


In [ ]:
sc.tl.leiden(adata, resolution=0.6)

We can now plot the results of the clustering:

In [ ]:
sc.pl.umap(adata, color=['leiden'])

---
## **8. Identifying cell types**

To identify the different cell types, we can visualise the expression of individual genes on the UMAP representation ...

In [ ]:
sc.pl.umap(adata, color=['leiden', 'IL7R', 'PPBP'])  


 ... or as violin plots across all clusters:

In [ ]:
sc.pl.violin(adata, ['IL7R', 'PPBP'], groupby='leiden')

#### **_Your turn_: Exercises**

__Exercise 1__: Shown below is a list of marker genes for different cell types. Can you identify which Cluster (0-7) corresponds to which cell type based on marker gene expression? Please fill in the table with your results by replacing the `?`. 

Cell type | Marker genes |  Cluster (0-7)
--- | --- |  ---
B cells | MS4A1 |  ?
CD4 T cells | IL7R |  ?
CD8 T cells | CD8A |  ?
NK cells | GNLY, NKG7 |  ?
Dendritic cells | FCER1A, CST3 |  ?
CD14 Monocytes | CD14, LYZ |  ?
FCGR3A Monocytes | FCGR3A, MS4A7 |  ?
Megakaryocytes | PPBP |  ?

**Exercise 2**: Create an array called `new_cluster_names` that contains the names of the different cell types in the order of the clusters (0-7). Then run the code below to plot your results!

In [ ]:
# Complete this:
new_cluster_names = ['', '', ...]

In [ ]:
# Then run this:
adata.rename_categories('leiden', new_cluster_names)
sc.pl.umap(adata, color='leiden', title='')

#**Well done!**



